In [2]:
import mlflow
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import pandas as pd
import re
import string
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import numpy as np

In [7]:
import nltk

In [3]:
df = pd.read_csv('IMDB.csv')
df = df.sample(500)
df.to_csv('data.csv', index=False)
df.head()

,review,sentiment
246,It's strange what fate does to some people. Wh...,negative
383,1985 was a good year for films - maybe even gr...,positive
750,It is enjoyable and fast-paced. <br /><br />Th...,negative
510,watch this movie. it's truly a good ride throu...,positive
99,"I am a Shakespeare fan, and I can appreciate w...",negative


In [ ]:
# data preprocessing

# Define text preprocessing functions
def lemmatization(text):
    """Lemmatize the text."""
    lemmatizer = WordNetLemmatizer()
    text = text.split()
    text = [lemmatizer.lemmatize(word) for word in text]
    return " ".join(text)

def remove_stop_words(text):
    """Remove stop words from the text."""
    stop_words = set(stopwords.words("english"))
    text = [word for word in str(text).split() if word not in stop_words]
    return " ".join(text)

def removing_numbers(text):
    """Remove numbers from the text."""
    text = ''.join([char for char in text if not char.isdigit()])
    return text

def lower_case(text):
    """Convert text to lower case."""
    text = text.split()
    text = [word.lower() for word in text]
    return " ".join(text)

def removing_punctuations(text):
    """Remove punctuations from the text."""
    text = re.sub('[%s]' % re.escape(string.punctuation), ' ', text)
    text = text.replace('؛', "")
    text = re.sub('\s+', ' ', text).strip()
    return text

def removing_urls(text):
    """Remove URLs from the text."""
    url_pattern = re.compile(r'https?://\s+|www\.\s+')
    return url_pattern.sub(r'', text)

def normalize_text(df):
    """Normalize the text data."""
    try:
        df['review'] = df['review'].apply(lower_case)
        df['review'] = df['review'].apply(remove_stop_words)
        df['review'] = df['review'].apply(removing_numbers)
        df['review'] = df['review'].apply(removing_punctuations)
        df['review'] = df['review'].apply(removing_urls)
        df['review'] = df['review'].apply(lemmatization)
        return df
    except Exception as e:
        print(f'Error during text normalization: {e}')
        raise

<>:32: SyntaxWarning: invalid escape sequence '\s'
<>:32: SyntaxWarning: invalid escape sequence '\s'
C:\Users\DELL\AppData\Local\Temp\ipykernel_17776\3798096103.py:32: SyntaxWarning: invalid escape sequence '\s'
  text = re.sub('\s+', ' ', text).strip()


In [8]:
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\DELL\AppData\Roaming\nltk_data...


True

In [9]:
df = normalize_text(df)
df.head()

,review,sentiment
246,strange fate people looking discount bin dvd r...,negative
383,good year film maybe even great one missing go...,positive
750,enjoyable fast paced br br way earth actor pla...,negative
510,watch movie truly good ride difficulty making ...,positive
99,shakespeare fan appreciate ken branagh done br...,negative


In [10]:
df['sentiment'].value_counts()

sentiment
negative    260
positive    240
Name: count, dtype: int64

In [11]:
x = df['sentiment'].isin(['positive','negative'])
df = df[x]

In [12]:
df['sentiment'] = df['sentiment'].map({'positive':1, 'negative':0})
df.head()

,review,sentiment
246,strange fate people looking discount bin dvd r...,0
383,good year film maybe even great one missing go...,1
750,enjoyable fast paced br br way earth actor pla...,0
510,watch movie truly good ride difficulty making ...,1
99,shakespeare fan appreciate ken branagh done br...,0


In [13]:
df.isnull().sum()

review       0
sentiment    0
dtype: int64

In [14]:
vectorizer = CountVectorizer(max_features=100)
X = vectorizer.fit_transform(df['review'])
y = df['sentiment']

In [15]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

In [17]:
import dagshub

mlflow.set_tracking_uri('https://dagshub.com/Parzival-Prime/MLOPs-Capstone-project.mlflow')
dagshub.init(repo_owner='Parzival-Prime', repo_name='MLOPs-Capstone-project', mlflow=True)

# mlflow.set_experiment("Logistic Regression Baseline")
mlflow.set_experiment("Logistic Regression Baseline")


Initialized MLflow to track repo "Parzival-Prime/MLOPs-Capstone-project"

Repository Parzival-Prime/MLOPs-Capstone-project initialized!

2025/05/19 18:13:39 INFO mlflow.tracking.fluent: Experiment with name 'Logistic Regression Baseline' does not exist. Creating a new experiment.


<Experiment: artifact_location='mlflow-artifacts:/4f40f7905319400189d12a715947269c', creation_time=1747658620319, experiment_id='0', last_update_time=1747658620319, lifecycle_stage='active', name='Logistic Regression Baseline', tags={}>

In [18]:
import mlflow
import logging
import os
import time
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Configure logging
logging.basicConfig(level=logging.INFO, format="%(asctime)s - %(levelname)s - %(message)s")

logging.info("Starting MLflow run...")

with mlflow.start_run():
    start_time = time.time()
    
    try:
        logging.info("Logging preprocessing parameters...")
        mlflow.log_param("vectorizer", "Bag of Words")
        mlflow.log_param("num_features", 100)
        mlflow.log_param("test_size", 0.25)

        logging.info("Initializing Logistic Regression model...")
        model = LogisticRegression(max_iter=1000)  # Increase max_iter to prevent non-convergence issues

        logging.info("Fitting the model...")
        model.fit(X_train, y_train)
        logging.info("Model training complete.")

        logging.info("Logging model parameters...")
        mlflow.log_param("model", "Logistic Regression")

        logging.info("Making predictions...")
        y_pred = model.predict(X_test)

        logging.info("Calculating evaluation metrics...")
        accuracy = accuracy_score(y_test, y_pred)
        precision = precision_score(y_test, y_pred)
        recall = recall_score(y_test, y_pred)
        f1 = f1_score(y_test, y_pred)

        logging.info("Logging evaluation metrics...")
        mlflow.log_metric("accuracy", accuracy)
        mlflow.log_metric("precision", precision)
        mlflow.log_metric("recall", recall)
        mlflow.log_metric("f1_score", f1)

        logging.info("Saving and logging the model...")
        mlflow.sklearn.log_model(model, "model")

        # Log execution time
        end_time = time.time()
        logging.info(f"Model training and logging completed in {end_time - start_time:.2f} seconds.")

        # Save and log the notebook
        # notebook_path = "exp1_baseline_model.ipynb"
        # logging.info("Executing Jupyter Notebook. This may take a while...")
        # os.system(f"jupyter nbconvert --to notebook --execute --inplace {notebook_path}")
        # mlflow.log_artifact(notebook_path)

        # logging.info("Notebook execution and logging complete.")

        # Print the results for verification
        logging.info(f"Accuracy: {accuracy}")
        logging.info(f"Precision: {precision}")
        logging.info(f"Recall: {recall}")
        logging.info(f"F1 Score: {f1}")

    except Exception as e:
        logging.error(f"An error occurred: {e}", exc_info=True)


2025-05-19 18:20:45,147 - INFO - Starting MLflow run...
2025-05-19 18:20:45,841 - WARNING - Retrying (Retry(total=6, connect=7, read=6, redirect=7, status=7)) after connection broken by 'RemoteDisconnected('Remote end closed connection without response')': /Parzival-Prime/MLOPs-Capstone-project.mlflow/api/2.0/mlflow/runs/create
2025-05-19 18:20:46,541 - INFO - Logging preprocessing parameters...
2025-05-19 18:20:47,919 - INFO - Initializing Logistic Regression model...
2025-05-19 18:20:47,923 - INFO - Fitting the model...
2025-05-19 18:20:48,057 - INFO - Model training complete.
2025-05-19 18:20:48,061 - INFO - Logging model parameters...
2025-05-19 18:20:48,474 - INFO - Making predictions...
2025-05-19 18:20:48,484 - INFO - Calculating evaluation metrics...
2025-05-19 18:20:48,534 - INFO - Logging evaluation metrics...
2025-05-19 18:20:50,825 - INFO - Saving and logging the model...
2025/05/19 18:21:27 WARNING mlflow.models.model: Model logged without a signature and input example. Pl

🏃 View run stylish-mare-605 at: https://dagshub.com/Parzival-Prime/MLOPs-Capstone-project.mlflow/#/experiments/0/runs/2eccbb98a99b4847badcc728972a5284
🧪 View experiment at: https://dagshub.com/Parzival-Prime/MLOPs-Capstone-project.mlflow/#/experiments/0
